In [229]:
import numpy as np    

### Importa libreria Scikit 

In [230]:
from sklearn.feature_extraction.text import TfidfVectorizer

### Importa librería NLTK para preprocesado

In [231]:
import nltk, string

# nltk.download('punkt') # if necessary...

In [232]:
from stop_words import get_stop_words

## Carga documentos del CORPUS

In [233]:
document = open('CORPUS.csv','r')
corpus = [doc.replace(';','').replace('\n','') for doc in document]

In [234]:
corpus.insert(0,'insertado en primera posición')

In [235]:
print(corpus)

['insertado en primera posición', 'Ley 22/2011, de 28 de julio, de residuos y suelos contaminados.', 'Ley 22/2011, de 28 de julio, de residuos y suelos no contaminados.', 'Orden PRA/321/2017, de 7 de abril, por la que se regulan los procedimientos de determinación de las emisiones de los contaminantes atmosféricos SO2, NOx, partículas y CO procedentes de las grandes instalaciones de combustión, el control de los instrumentos de medida y el tratamiento y remisión de la información relativa a dichas emisiones.', 'Real Decreto 198/2017, de 3 de marzo, por el que se modifican el Real Decreto 1728/2007, de 21 de diciembre, por el que se establece la normativa básica de control que deben cumplir los operadores del sector lácteo y se modifica el Real Decreto 217/2004, de 6 de febrero, por el que se regulan la identificación y registro de los agentes, establecimientos y contenedores que intervienen en el sector lácteo, y el registro de los movimientos de la leche, el Real Decreto 752/2011, de 

## Preprocesado

### Stop Words, Stemmization, Tokenization, LowerCase, and Remove Punctuation

In [236]:
additional_stop_words = ['https','-','amp','&amp;']
stop_words = get_stop_words('english') \
    + get_stop_words('spanish')  \
    + additional_stop_words

In [237]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

In [238]:
def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words= stop_words )

# def cosine_sim(text1, text2):
#     tfidf = vectorizer.fit_transform([text1, text2])
#     return ((tfidf * tfidf.T).A)[0,1]

## Procesado

In [239]:
tfidf = vectorizer.fit_transform(corpus)

In [240]:
pairwise_similarity = (tfidf * tfidf.T)
arr = pairwise_similarity.A

In [241]:
pairwise_similarity

<35x35 sparse matrix of type '<class 'numpy.float64'>'
	with 825 stored elements in Compressed Sparse Row format>

In [242]:
arr

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.05200758],
       [0.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.05200758],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.05200758, 0.05200758, ..., 0.        , 0.        ,
        1.        ]])

In [243]:
### elimina la coincidencia de un doc consigo mismo      
np.fill_diagonal(arr, np.nan)                                                                                              

In [244]:
### Busca índice del doc del que vamos a buscar otro similar

In [245]:
query_doc = "Ley 22/2011, de 28 de julio, de residuos y suelos contaminados."          

input_idx = corpus.index(query_doc)    

### Imprime el índice del doc del que vamos a buscar otro similar

In [246]:
input_idx

1

### Encuentra el índice del doc con mayor similaridad

In [247]:
result_idx = np.nanargmax(arr[input_idx]) 

In [248]:
result_idx

2

### Coeficiente de similaridad

In [249]:
arr[input_idx,result_idx]

0.9999999999999999

### Imprime el doc con el coeficiente de mayor similaridad

In [250]:
  corpus[result_idx]    

'Ley 22/2011, de 28 de julio, de residuos y suelos no contaminados.'

In [251]:
### Obtiene los índices de los 5 docs que más similitud tienen con el de entrada

In [252]:
idx = (-arr[input_idx]).argsort()[:5]

In [253]:
idx

array([ 2, 31,  9, 25, 21])

In [254]:
### Obtiene los coeficientes de similaridad de los 5 docs más similares

In [255]:
[ arr[input_idx, index] for index in idx ]

[0.9999999999999999,
 0.2814987344752851,
 0.17645184381147244,
 0.1672440655512475,
 0.1033249205060555]

In [256]:
### Lista los 5 docs con mayor similaridad al de entrada

In [257]:
[corpus[index] for index in idx]

['Ley 22/2011, de 28 de julio, de residuos y suelos no contaminados.',
 'Real Decreto 9/2005, de 14 de enero, por el que se establece la relación de actividades potencialmente contaminantes del suelo y los criterios y estándares para la declaración de suelos contaminados.',
 'Ley 21/2015, de 20 de julio, por la que se modifica la Ley 43/2003, de 21 de noviembre, de Montes.',
 'Ley 4/2012, de 28 de diciembre, de Medidas Financieras y Administrativas de la Comunidad Autónoma de Extremadura.',
 'Real Decreto 815/2013, de 18 de octubre, por el que se aprueba el Reglamento de emisiones industriales y de desarrollo de la Ley 16/2002, de 1 de julio, de prevención y control integrados de la contaminación.']

# REGEX

In [258]:
import re

In [259]:
### Recoge títulos de Items de la BBDD
items_file = open('items_BBDD.txt','r')

In [260]:
items_BBDD = items_file.read()

In [261]:
print(items_BBDD)

Decisión 2000/532/CE,
Directiva 1999/45/CE,
Directiva 2006/121/CE,
Directiva 2008/98/CE,
Directiva 67/548/CE,
Ley 10/1998,
Ley 15/1998,
Ley 11/1997,
Ley 11/2007,
Ley 16/2002,
Ley 17/2009,
Ley 22/2011,
Ley 26/2007,
Ley 27/2006,
Ley 30/1992,
Ley 30/2007,
Ley 34/2007,
Ley 40/2010,
Ley 50/2010,
Ley 9/2006,
Real Decreto 1419/2005,
Real Decreto 2090/2008,
Real Decreto 975/2009,
Reglamento (CE) N.º 1013/2006,
Reglamento (CE) n.º 1013/2006,
Reglamento (CE) n.º 1069/2009,
Reglamento (CE) n.º 1272/2008,
Reglamento (CE) n.º 1418/2007,
Reglamento (CE) n.º 1774/2002,
Reglamento (CE) n.º 1907/2006,
Reglamento (CE) n.º 440/2008


In [262]:
items_BBDD = items_BBDD.split(',\n')

In [263]:
file = open('texto.txt','r')
#corpus = [doc.replace(';','').replace('\n','') for doc in document]

In [264]:
#print (file.read())

In [265]:
texto = file.read()
#texto.replace('\n','')
print(texto)

pladas en el artículo 15
Medidas que pueden afectar a las condiciones marco de la generación de residuos

1. La aplicación de medidas de planificación u otros instrumentos económicos que fomenten una utilización eficiente de los recursos.

2. La promoción de la investigación y el desarrollo destinados a obtener tecnologías y productos más limpios y con menos residuos, así como la difusión y utilización de los resultados de estos trabajos de investigación y desarrollo.

3. La elaboración de indicadores significativos y efectivos de las presiones medioambientales relacionadas con la generación de residuos con miras a contribuir a la prevención de la generación de residuos a todos los niveles, desde las comparaciones de productos a escala comunitaria hasta las intervenciones por parte de las autoridades locales o medidas de carácter nacional.

Medidas que pueden afectar a la fase de diseño, producción y distribución

4. La promoción del eco-diseño (la integración sistemática de los aspect

In [266]:
# Ley 22/2011
pattern = 'Ley [0-9]+\/[0-9]+'
result = re.findall(pattern, texto, flags=re.IGNORECASE)

In [267]:
## Elimina duplicados
list(set(result))

['Ley 17/2009',
 'Ley 26/2007',
 'Ley 34/2007',
 'Ley 40/2010',
 'Ley 30/1992',
 'Ley 30/2007',
 'Ley 11/1997',
 'Ley 9/2006',
 'Ley 27/2006',
 'Ley 16/2002',
 'Ley 11/2007',
 'Ley 22/2011',
 'Ley 10/1998']

In [268]:
# Real Decreto 975/2009
pattern = 'Real Decreto [0-9]+\/[0-9]+'
result = re.findall(pattern, texto, flags=re.IGNORECASE)

In [269]:
## Elimina duplicados
list(set(result))

['Real Decreto 975/2009', 'Real Decreto 1419/2005', 'Real Decreto 2090/2008']

In [270]:
# Directiva 2008/98/CE
pattern = 'Directiva [0-9]+\/[0-9]+\/CE'
result = re.findall(pattern, texto, flags=re.IGNORECASE)

In [271]:
## Elimina duplicados
list(set(result))

['Directiva 2008/98/CE',
 'Directiva 2006/121/CE',
 'Directiva 1999/45/CE',
 'Directiva 67/548/CE']

In [272]:
# Reglamento (CE) n.º 1013/2006
pattern = 'Reglamento \(CE\) n\.º [0-9]+\/[0-9]+'
result = re.findall(pattern, texto, flags=re.IGNORECASE)

In [273]:
## Elimina duplicados
list(set(result))

['Reglamento (CE) n.º 1418/2007',
 'Reglamento (CE) n.º 1907/2006',
 'Reglamento (CE) n.º 1069/2009',
 'Reglamento (CE) n.º 1272/2008',
 'Reglamento (CE) n.º 1013/2006',
 'Reglamento (CE) n.º 1774/2002',
 'Reglamento (CE) n.º 440/2008',
 'Reglamento (CE) N.º 1013/2006']

In [274]:
# Decisión 2000/532/CE
pattern = 'Decisión [0-9]+\/[0-9]+\/CE'
result = re.findall(pattern, texto, flags=re.IGNORECASE)

In [275]:
## Elimina duplicados
list(set(result))

['Decisión 2000/532/CE']

In [276]:
# Todas las posibilidades a la
pattern = 'Ley [0-9]+\/[0-9]+|Real Decreto [0-9]+\/[0-9]+|Directiva [0-9]+\/[0-9]+\/CE|Reglamento \(CE\) n\.º [0-9]+\/[0-9]+|Decisión [0-9]+\/[0-9]+\/CE'
result = re.findall(pattern, texto, flags=re.IGNORECASE)

In [277]:
## Elimina duplicados y ordena lista
result = list(set(result))
result.sort()

In [278]:
result

['Decisión 2000/532/CE',
 'Directiva 1999/45/CE',
 'Directiva 2006/121/CE',
 'Directiva 2008/98/CE',
 'Directiva 67/548/CE',
 'Ley 10/1998',
 'Ley 11/1997',
 'Ley 11/2007',
 'Ley 16/2002',
 'Ley 17/2009',
 'Ley 22/2011',
 'Ley 26/2007',
 'Ley 27/2006',
 'Ley 30/1992',
 'Ley 30/2007',
 'Ley 34/2007',
 'Ley 40/2010',
 'Ley 9/2006',
 'Real Decreto 1419/2005',
 'Real Decreto 2090/2008',
 'Real Decreto 975/2009',
 'Reglamento (CE) N.º 1013/2006',
 'Reglamento (CE) n.º 1013/2006',
 'Reglamento (CE) n.º 1069/2009',
 'Reglamento (CE) n.º 1272/2008',
 'Reglamento (CE) n.º 1418/2007',
 'Reglamento (CE) n.º 1774/2002',
 'Reglamento (CE) n.º 1907/2006',
 'Reglamento (CE) n.º 440/2008']

In [279]:
#pattern = '(\w+)\s+(\w+)\s+[0-9]+\/[0-9]+'
#re.findall(pattern, texto, flags=re.IGNORECASE)

In [280]:
# Recoge items de la BBDD y los incluye en un set inmutable (frozenset)
items_set = frozenset(items_BBDD)

In [281]:
## Busca coincidencias entre los items recogidos en BOE actual y la BBDD de items

In [282]:
items_set.intersection(result)

frozenset({'Decisión 2000/532/CE',
           'Directiva 1999/45/CE',
           'Directiva 2006/121/CE',
           'Directiva 2008/98/CE',
           'Directiva 67/548/CE',
           'Ley 10/1998',
           'Ley 11/1997',
           'Ley 11/2007',
           'Ley 16/2002',
           'Ley 17/2009',
           'Ley 22/2011',
           'Ley 26/2007',
           'Ley 27/2006',
           'Ley 30/1992',
           'Ley 30/2007',
           'Ley 34/2007',
           'Ley 40/2010',
           'Ley 9/2006',
           'Real Decreto 1419/2005',
           'Real Decreto 2090/2008',
           'Real Decreto 975/2009',
           'Reglamento (CE) N.º 1013/2006',
           'Reglamento (CE) n.º 1013/2006',
           'Reglamento (CE) n.º 1069/2009',
           'Reglamento (CE) n.º 1272/2008',
           'Reglamento (CE) n.º 1418/2007',
           'Reglamento (CE) n.º 1774/2002',
           'Reglamento (CE) n.º 1907/2006',
           'Reglamento (CE) n.º 440/2008'})

In [283]:
set(items_BBDD).intersection(result)

{'Decisión 2000/532/CE',
 'Directiva 1999/45/CE',
 'Directiva 2006/121/CE',
 'Directiva 2008/98/CE',
 'Directiva 67/548/CE',
 'Ley 10/1998',
 'Ley 11/1997',
 'Ley 11/2007',
 'Ley 16/2002',
 'Ley 17/2009',
 'Ley 22/2011',
 'Ley 26/2007',
 'Ley 27/2006',
 'Ley 30/1992',
 'Ley 30/2007',
 'Ley 34/2007',
 'Ley 40/2010',
 'Ley 9/2006',
 'Real Decreto 1419/2005',
 'Real Decreto 2090/2008',
 'Real Decreto 975/2009',
 'Reglamento (CE) N.º 1013/2006',
 'Reglamento (CE) n.º 1013/2006',
 'Reglamento (CE) n.º 1069/2009',
 'Reglamento (CE) n.º 1272/2008',
 'Reglamento (CE) n.º 1418/2007',
 'Reglamento (CE) n.º 1774/2002',
 'Reglamento (CE) n.º 1907/2006',
 'Reglamento (CE) n.º 440/2008'}